# Data Visualization: Week 2, Lecture 1

#### Learning Objectives
By the end of this CodeAlong, students will be able to:
- Perform and interpret the results of the augmented Dickey-Fuller test
- Identify the necessary order of differencing to achieve stationarity
- Plot ACF and PACF plots
- Fit and evaluate an ARIMA model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import set_config

from pmdarima.model_selection import train_test_split
from pmdarima.arima.utils import ndiffs

import statsmodels.tsa.api as tsa

In [ ]:
# set_config(transform_output="pandas")
# plt.rcParams["figure.figsize"] = (12, 4)
# sns.set_context("talk", font_scale=0.9)

## Custom functions

These functions are imported from the LP.

In [ ]:
def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False) 
    r_squared = r2_score(y_true, y_pred)
    if verbose == True:
        # Print Result with Label and Header
        header = "-"*60
        print(header, f"Regression Metrics: {label}", header, sep='\n')
        print(f"- MAE = {mae:,.3f}")
        print(f"- MSE = {mse:,.3f}")
        print(f"- RMSE = {rmse:,.3f}")
        print(f"- R^2 = {r_squared:,.3f}")
    if output_dict == True:
        metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
        return metrics

def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
    # Get predictions for training data
    y_train_pred = reg.predict(X_train)

    # Call the helper function to obtain regression metrics for training data
    results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
    print()
    # Get predictions for test data
    y_test_pred = reg.predict(X_test)
    # Call the helper function to obtain regression metrics for test data
    results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )

    # Store results in a dataframe if ouput_frame is True
    if output_frame:
        results_df = pd.DataFrame([results_train,results_test])
        # Set the label as the index 
        results_df = results_df.set_index('Label')
        # Set index.name to none to get a cleaner looking result
        results_df.index.name=None
        # Return the dataframe
        return results_df.round(3)
    
# Custom function for Ad Fuller Test
def get_adfuller_results(ts, alpha=.05, label='adfuller', **kwargs): #kwargs for adfuller()
    # Saving each output
    (test_stat, pval, nlags, nobs, crit_vals_d, 
    icbest ) = tsa.adfuller(ts, **kwargs)
    # Converting output to a dictionary with the interpretation of p
    adfuller_results = {'Test Statistic': test_stat,
                        "# of Lags Used":nlags, 
                       '# of Observations':nobs,
                        'p-value': round(pval,6),
                        'alpha': alpha,
                       'sig/stationary?': pval < alpha}
    return pd.DataFrame(adfuller_results, index =[label])

## Importing data & exploratory analysis

In [ ]:
# sunscreen
df = pd.read_csv('sunscreen_popularity.txt', skiprows= [0])
df.head()

In [ ]:
# Visualize the data
ax = df.plot()
ax.set(ylabel="Popularity", xlabel="Time", title="Google Trends popularity for sunscreen");

In [ ]:
df.head(3)

#### Set month to index, set frequency, and check for nulls

In [ ]:
# cast month as date time, set index, set freq, check nulls




#### Train-test split with test_size=0.05, check dimensions, and plot train & test

In [ ]:
# train-test split


In [ ]:
# plot train and test


## Testing for stationarity

Recall that we use the augmented Dickey-Fuller test to evaluate whether our data are stationary.

If the p-value is less than alpha, we reject the null hypothesis of non-stationarity. Otherwise we fail to reject.

### Use the ADF to evaluate for stationarity

In [ ]:
# use get_adfuller_results


### Use the ADF to evaluate whether once-differenced data are stationary

In [ ]:
# use get_adfuller_results


### Use `ndiffs` to check order for making data stationary

In [ ]:
# use ndiffs


### Plot the differenced data as well as ACF and PACF

In [ ]:
# plot differenced data


In [ ]:
# plot ACF


In [ ]:
# plot PACF


## Fitting an ARIMA model

Remember that identifying the orders of an ARIMA model can be challenging when both p and q are non-zero.

There are also seasonal trends in this dataset. Our linear ARIMA model may not capture those patterns well!

Let's start with a model of order (2,1,2).

In [ ]:
# First define the orders (p,d,q)
p = 
d = 
q = 

# Now instantiate the model with the data and fit
model = 

## Evaluating the ARIMA model

Check the model summary.

Store the model forecast using `model.get_forecast()` as `preds_df`

In [ ]:
model.summary()

In [ ]:
# use model.get_forecast()
# what does summary_frame() do? why might this be useful?


## Check forecast by plotting and calculating evaluation metrics

### Additional custom functions

In [ ]:
def plot_forecast(ts_train, ts_test, forecast_df, n_train_lags=None, 
                  figsize=(10,4), title='Comparing Forecast vs. True Data'):
    ### PLot training data, and forecast (with upper/,lower ci)
    fig, ax = plt.subplots(figsize=figsize)
    # setting the number of train lags to plot if not specified
    if n_train_lags==None:
        n_train_lags = len(ts_train)
            
    # Plotting Training  and test data
    ts_train.iloc[-n_train_lags:].plot(ax=ax, label="train")
    ts_test.plot(label="test", ax=ax)
    # Plot forecast
    forecast_df['mean'].plot(ax=ax, color='green', label="forecast")
    # Add the shaded confidence interval
    ax.fill_between(forecast_df.index, 
                    forecast_df['mean_ci_lower'],
                   forecast_df['mean_ci_upper'],
                   color='green', alpha=0.3,  lw=2)
    # set the title and add legend
    ax.set_title(title)
    ax.legend();
    
    return fig, ax



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

def regression_metrics_ts(ts_true, ts_pred, label="", verbose=True, output_dict=False,):
    # Get metrics
    mae = mean_absolute_error(ts_true, ts_pred)
    mse = mean_squared_error(ts_true, ts_pred)
    rmse = mean_squared_error(ts_true, ts_pred, squared=False)
    r_squared = r2_score(ts_true, ts_pred)
    mae_perc = mean_absolute_percentage_error(ts_true, ts_pred) * 100

    if verbose == True:
        # Print Result with label
        header = "---" * 20
        print(header, f"Regression Metrics: {label}", header, sep="\n")
        print(f"- MAE = {mae:,.3f}")
        print(f"- MSE = {mse:,.3f}")
        print(f"- RMSE = {rmse:,.3f}")
        print(f"- R^2 = {r_squared:,.3f}")
        print(f"- MAPE = {mae_perc:,.2f}%")

    if output_dict == True:
        metrics = {
            "Label": label,
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R^2": r_squared,
            "MAPE(%)": mae_perc,
        }
        return metrics

### Use the custom functions to plot the forecast, display the evaluation metrics, and plot model diagnostics

In [ ]:
# use plot_forecast()



In [ ]:
# use regression_metrics_ts



In [ ]:
# use model.plot_diagnostics()

